In [9]:
import numpy as np
import pandas as pd
import os, sys

import json

import plotly.graph_objects as go

In [10]:
data_dir = "/mnt/c/Users/Public/Documents/Uni/master/masterarbeit/ampel/ampel-results/random/TransientTable"
save_dir = "./pics/random/"
os.makedirs(save_dir, exist_ok=True)
data_files = [file for file in os.listdir(data_dir) if ".csv" in file and "~lock" not in file]

total_dataframe = pd.DataFrame()
for datafile in data_files:
    tmp_df = pd.read_csv(os.path.join(data_dir, datafile), sep=";")
    total_dataframe = pd.concat((total_dataframe, tmp_df))


total_entries = len(total_dataframe)

In [11]:
def jsonify(string: str):
    if type(string) != str:
        return ""
    tmp_str = string.replace("'", '"')
    tmp_str = tmp_str.replace("None", "null")
    tmp_str = tmp_str.replace("True", "true")
    tmp_str = tmp_str.replace("False", "false")

    return tmp_str

## CHISQUARE STUFF

In [12]:


#print(total_dataframe.head())


pos_rowslist_dict = {"bns_maurice_1": [], "bns_maurice_2": [], "bns_maurice_3": [], "bns_maurice_4": []}
total_dataframe["ndet"] = np.zeros(len(total_dataframe))
rowslist = []
ndet_list = []
for k, row in total_dataframe.iterrows():
    dec_str = jsonify(row["rise_decline"])
    #print(tmp_str)
    dec_dict = json.loads(dec_str)
    #print(dec_dict["ndet"], row["kilonovaness"])

    ndet_list.append(dec_dict["ndet"])

    rowslist.append(dec_dict)

    #print(row["chisquares"])
    pos_str = jsonify(row["chisquares"])
    if pos_str != "":
        pos_dict = json.loads(pos_str)
        #print(pos_dict)
        for key, value in pos_dict.items():
            value["ndet"] = dec_dict["ndet"]
            value["latest_det"] = dec_dict["jd_last"]
            value["first_det"] = dec_dict["jd_det"]
            value["kilonovaness"] = row["kilonovaness"]
            value["map_seed"] = row["map_seed"]
            value["map_dist"] = row["map_dist"]
            value["map_area"] = row["map_area"]
            if value["ndof"] > 0:
                value["red_chisqu"] = value["chisq"] / value["ndof"]
            else:
                value["red_chisqu"] = -999
            pos_rowslist_dict[key].append(value)
    #print(row["rise_decline"]["ndet"])

total_dataframe["ndet"] = ndet_list

print(rowslist[0])
print(len(rowslist))
pos_df_dict = {}
for key in pos_rowslist_dict:
    pos_df_dict[key] = pd.DataFrame(pos_rowslist_dict[key]).sort_values(by="ndet")

print(len(pos_df_dict[key]))

{'ndet': 1, 'frac_pos': 1.0, 'jd_det': 2458735.0098727, 'jd_last': 2458735.0098727, 't_lc': 0.0, 't_predetect': None, 'mag_det': 19.657461166381836, 'band_det_id': 64, 'mag_last': 19.657461166381836, 'band_last_id': 64, 'mag_min': 19.657461166381836, 'bool_pure': True, 'jd_peak_ztfg': 2458735.0098727, 'bool_rise': False, 'bool_fall': False, 'bool_peaked': False, 'bool_fastrise': None, 'bool_fastfall': None, 'bool_hasgaps': None, 'det_bands': 64, 'last_bands': 64, 'success': True}
4781
1959


In [13]:
total_dataframe[["kilonovaness", "ndet", "rise_decline"]]

,kilonovaness,ndet,rise_decline
0,8,1,"{'ndet': 1, 'frac_pos': 1.0, 'jd_det': 2458735..."
1,8,1,"{'ndet': 1, 'frac_pos': 1.0, 'jd_det': 2458735..."
2,7,13,"{'ndet': 13, 'frac_pos': 1.0, 'jd_det': 245871..."
3,6,2,"{'ndet': 2, 'frac_pos': 1.0, 'jd_det': 2458736..."
4,5,6,"{'ndet': 6, 'frac_pos': 1.0, 'jd_det': 2458725..."
...,...,...,...
4,12,5,"{'ndet': 5, 'frac_pos': 1.0, 'jd_det': 2459314..."
5,12,5,"{'ndet': 5, 'frac_pos': 1.0, 'jd_det': 2459314..."
6,10,3,"{'ndet': 3, 'frac_pos': 1.0, 'jd_det': 2459316..."
7,5,5,"{'ndet': 5, 'frac_pos': 1.0, 'jd_det': 2459314..."


In [14]:
keys = list(pos_df_dict.keys())
key = keys[0]

fig = go.Figure()

max_red_chisq = 100
n_bins = 10
bins = np.linspace(0.8, max_red_chisq, n_bins)
print("bins:", bins)

os.makedirs(save_dir, exist_ok=True)
for key in keys[:]:

    total_number = len(pos_df_dict[key])
    print(f"% entries with fit:", key, ": ", total_number / total_entries)
    above_0_ndof = len(pos_df_dict[key][pos_df_dict[key]["ndof"] > 0])
    percentage_above_0_ndof = above_0_ndof / total_entries * 100
    print("ndof > 0:", above_0_ndof,"is",  percentage_above_0_ndof, "%")

    tmp_data = pos_df_dict[key].copy()[np.logical_and(pos_df_dict[key]["chisq"] <= max_red_chisq, pos_df_dict[key]["chisq"] >= 0.8) ]

    tmp_hist = np.histogram(tmp_data["red_chisqu"], bins=bins)
    #print(tmp_hist)

    fig.add_trace(go.Bar(x=bins, y = tmp_hist[0]/total_entries*100, name=key))
    #fig.add_trace(go.Histogram(x=tmp_data["chisq"], name=key))
fig.update_layout({"xaxis_title": "reduced chisqu",
                   "yaxis_title": "Portion of all candidates (%)",
                   "title": "Reduced chisqu. distribution over random maps",
                   "width": 600})
fig.show()

filename = os.path.join(save_dir, "chisqu_dist.pdf")
fig.write_image(filename)

bins: [  0.8         11.82222222  22.84444444  33.86666667  44.88888889
  55.91111111  66.93333333  77.95555556  88.97777778 100.        ]
% entries with fit: bns_maurice_1 :  0.40974691487136583
ndof > 0: 830 is 17.360384856724533 %
% entries with fit: bns_maurice_2 :  0.40974691487136583
ndof > 0: 830 is 17.360384856724533 %
% entries with fit: bns_maurice_3 :  0.40974691487136583
ndof > 0: 830 is 17.360384856724533 %
% entries with fit: bns_maurice_4 :  0.40974691487136583
ndof > 0: 830 is 17.360384856724533 %


## NDET STUFF

In [15]:
ndet_hist = np.histogram(pos_df_dict[key]["ndet"], bins=np.max(pos_df_dict[key]["ndet"]))

fig = go.Figure()

fig.add_trace(go.Scatter(x=np.arange(np.max(pos_df_dict[key]["ndet"])), y=ndet_hist[0]/len(pos_df_dict[key])))
fig.update_layout({"xaxis_title": "Number of detections",
                   "yaxis_title": "Counts (relative)",
                   "xaxis_range": (0, 25),
                   "width": 500,
                   "title": "Number of detections in random map events"})
fig.show()
filename = os.path.join(save_dir, "ndet.pdf")
fig.write_image(filename)

In [16]:
import plotly.express as px

In [21]:


fig = px.density_heatmap(x=total_dataframe["ndet"], y=total_dataframe["kilonovaness"], text_auto=True)
fig.update_traces(
    marker=dict(size=2,
                              line=dict(width=2,
                                        color='DarkSlateGrey')),
                  selector=dict(mode='markers'),
                  
                  )
fig.update_layout({"xaxis_title": "ndet",
                   "yaxis_title": "kilonovaness"})

fig.show()
print(len(total_dataframe["ndet"]))

4781


### MEAN NDET IN RANDOM MAPS

In [18]:
maps = total_dataframe["map_seed"].unique()

mean_ndet_list = []
std_ndet_list = []
median_ndet_list = []
mean_kilonovaness_list = []
std_kilonovaness_list = []
max_kilonovaness_list = []
median_kilonovaness_list = []

for k, map in enumerate(maps):
    tmp_df = total_dataframe.copy()[total_dataframe["map_seed"] == map]

    mean_ndet_list.append(np.mean(tmp_df["ndet"]))
    median_ndet_list.append(np.median(tmp_df["ndet"]))
    std_ndet_list.append(np.std(tmp_df["ndet"]))

    mean_kilonovaness_list.append(np.mean(tmp_df["kilonovaness"]))
    std_kilonovaness_list.append(np.std(tmp_df["kilonovaness"]))
    max_kilonovaness_list.append(np.max(tmp_df["kilonovaness"]))
    median_kilonovaness_list.append(np.median(tmp_df["kilonovaness"]))

ndet_df = pd.DataFrame(
    {
        "map": maps,
        "mean_ndet": mean_ndet_list,
        "std_ndet": std_ndet_list,
        "median_ndet": median_ndet_list,
        "mean_kilonovaness": mean_kilonovaness_list,
        "max_kilonovaness": max_kilonovaness_list,
        "median_kilonovaness": median_kilonovaness_list,
        "std_kilonovaness": std_kilonovaness_list
    }
)

ndet_df

,map,mean_ndet,std_ndet,median_ndet,mean_kilonovaness,max_kilonovaness,median_kilonovaness,std_kilonovaness
0,1005342624,1.512195,2.001487,1.0,2.926829,8,3.0,1.892092
1,1007319134,1.000000,0.000000,1.0,3.000000,3,3.0,0.000000
2,1017321636,12.312500,5.902529,15.0,7.312500,10,7.0,1.445629
3,1021381495,2.500000,0.500000,2.5,12.000000,12,12.0,0.000000
4,1022624203,1.882353,3.230656,1.0,3.784314,12,3.0,2.703039
...,...,...,...,...,...,...,...,...
251,977416226,4.750000,2.165064,6.0,8.000000,13,9.0,5.196152
252,988314718,4.000000,2.000000,4.0,8.500000,9,8.5,0.500000
253,988570821,5.000000,0.000000,5.0,2.000000,2,2.0,0.000000
254,997883860,6.400000,7.269113,2.5,1.800000,4,1.5,0.979796


In [19]:
fig = go.Figure()

fig.add_trace(go.Histogram(x=ndet_df["mean_ndet"]))

In [20]:
fig = go.Figure()

fig.add_trace(go.Scatter(y=ndet_df["mean_kilonovaness"], x=ndet_df["mean_ndet"], mode="markers", name="mean kilonovaness"))
fig.add_trace(go.Scatter(y=ndet_df["max_kilonovaness"], x=ndet_df["mean_ndet"], mode="markers", name="max kilonovaness", opacity=0.4))